# <div style="padding:20px;color:white;margin:0;font-size:200%;text-align:center;display:fill;border-radius:5px;background-color:#AF601A;overflow:hidden;font-weight:500">TPS June 2022
</div>




 ### As some of the Kagglers suggested, subset F1 (columns starting with F_1) and subset F3 (columns starting with F_3) might be time series data. Please see my other notebook where both F1 and F3 are treated as non-time series data in the public [notebook](https://www.kaggle.com/code/abdulravoofshaik/quick-eda-and-missing-values-tutorial). 
 ### The following cartoon depicts how we can use past information to identify the missing data/ predict future data.


<img src="https://miro.medium.com/max/1400/0*bCS3EWiVfLIZqwIW.gif">



<div style="color:white;display:fill;border-radius:8px;
            background-color:#E59866;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:black;"><b>1.0 | Load data and Preprocessing</b></p>
</div>

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm
import missingno as msno
pd.set_option('display.max_columns', None)
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
import matplotlib.colors
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.plotting import autocorrelation_plot
from pandas.plotting import lag_plot
from statsmodels.tsa.stattools import adfuller, acf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [ ]:
data = pd.read_csv("../input/tabular-playground-series-jun-2022/data.csv")
Target = pd.read_csv("../input/tabular-playground-series-jun-2022/sample_submission.csv", index_col='row-col')

In [ ]:
## we apply the same concept to individual column and develop 80 individual models. As we have noticed earlier, dataset has four different subsets. Lets divide them.
# this code snippet is taken from https://www.kaggle.com/code/martynovandrey/tps-jun-22-splitted-dataset-24x-faster. Consider upvoting the original author also
features = list(data.columns)
features_1, features_2, features_3, features_4 = [], [], [], []
F = [[], [], [], [], []]
for feature in features:
    for i in [1, 2, 3, 4]:
        if feature.split('_')[1] == str(i):
            F[i].append(feature)
df = [[], [], [], [], []]

for i in [1, 2, 3, 4]:
    df[i] = data[F[i]]
    corr = df[i].corr()

### Columns starting with F_4 are correlated with each other, we do not see such pattern in other subsets.

In [ ]:
# lets take look at F1 first
F1_df=df[1]
F1_df.head()


In [ ]:
# for the purpose of simpicity I am using only first 10000 rows of F1 dataset, u can increase n if you want to see full picture
n=10000
F1_df=F1_df.iloc[:n,:]
F1_df.head(3)

In [ ]:
for column in F1_df.columns: 
    F1_df[column].fillna(value=F1_df[column].mean(), inplace=True)

In [ ]:
for column in F1_df.columns: 
    print('Current timestep data vs previous timestep data for Colunm Name : ', column)
    plt.figure()
    lag_plot(F1_df[column])
    plt.show()
    
    

### Based on the above figures, no pattern is noticed between current timestep vs previous timestep information. There might not be any correlation at lag=1, but maybe there is a correlation at lag=50. Let's make an autocorrelation plot to compare the values of the autocorrelation function (ACF) against different lag sizes. 

In [ ]:
# for column in F1_df.columns: 
#     print('Processing Colunm Name : ', column)
#     F1_df[column].fillna(value=F1_df[column].mean(), inplace=True)
#     plt.clf()
#     fig, ax = plt.subplots(figsize=(5, 5))
#     autocorrelation_plot(F1_df[column], ax=ax)    
#     plt.show()

In [ ]:
for column in F1_df.columns: 
    print('Plotting ACF vs lag size for colunm name : ', column)
    plt.clf()
    fig, ax = plt.subplots(figsize=(40, 10))
    plot_acf(F1_df[column], lags=100, use_vlines=False, ax=ax)    
    plt.show()

### Based on the above plots, we see that values for the ACF are within 95% confidence interval for lags > 0.  which verifies that our data doesn’t have any autocorrelation. Is my assumption correct? Please let me know in comments. 
### May be the data is correlated at first-order difference time-series data?? Lets check with one of the column 'F_1_2' <br>

In [ ]:
ddiff = F1_df['F_1_2'].diff(1)[1:]
ddiff.plot()

In [ ]:
fig.clf()
fig, ax = plt.subplots(figsize=(10, 10))
print('Current timestep data (frist order)  vs previous timestep data (frist order) for column F_1_2')
lag_plot(ddiff, ax=ax)

In [ ]:
fig.clf()
fig, ax = plt.subplots(figsize=(15, 8))
autocorrelation_plot(ddiff, ax=ax)

### Based on the above plot, the ACF for first order data of the selected column  doesn’t have any autocorrelation. Is my assumption correct? Please let me know in comments. 

### Next steps, try for subset F3. work in progress

<div style="color:white;display:fill;border-radius:8px;
            background-color:#E59866;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:black;"><b>3.0 | References</b></p>
</div>

https://towardsdatascience.com/using-the-missingno-python-library-to-identify-and-visualise-missing-data-prior-to-machine-learning-34c8c5b5f009 <br>
https://towardsdatascience.com/how-to-handle-missing-data-8646b18db0d4 <br>
https://www.kaggle.com/code/parulpandey/a-guide-to-handling-missing-values-in-python/notebook <br>
https://www.kaggle.com/code/residentmario/using-missingno-to-diagnose-data-sparsity/notebook <br>
https://www.analyticsvidhya.com/blog/2021/05/dealing-with-missing-values-in-python-a-complete-guide/ <br>
https://www.kaggle.com/code/calebreigada/getting-started-eda-preprocessing <br>
https://medium.com/swlh/impute-missing-values-the-right-way-c63735fccccd <br>
https://www.kaggle.com/code/jeegarmaru/timeseriesanalysis-autocorrelation/notebook <br>

